In [1]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (9,549 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [2]:
!mkdir fonts output train

In [14]:
font_name = 'TypoUpright BT'  #@param {type: "string"}



In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
!chmod 755 -R /content/drive/MyDrive/tesseract/src/training/tesstrain.sh

In [17]:
!combine_tessdata -e /content/drive/MyDrive/tesseract/tessdata/eng.traineddata eng.lstm

Extracting tessdata components from /content/drive/MyDrive/tesseract/tessdata/eng.traineddata
Wrote eng.lstm
Version string:4.00.00alpha:eng:synth20170629:[1,36,0,1Ct3,3,16Mp3,3Lfys64Lfx96Lrx96Lfx512O1c1]
17:lstm:size=11689099, offset=192
18:lstm-punc-dawg:size=4322, offset=11689291
19:lstm-word-dawg:size=3694794, offset=11693613
20:lstm-number-dawg:size=4738, offset=15388407
21:lstm-unicharset:size=6360, offset=15393145
22:lstm-recoder:size=1012, offset=15399505
23:version:size=80, offset=15400517


In [19]:
%%bash
# Ensure necessary langdata files are present

# Download eng.config if not present
if [ ! -f /content/drive/MyDrive/langdata_lstm/eng/eng.config ]; then
    echo "Downloading eng.config..."
    wget -O /content/drive/MyDrive/langdata_lstm/eng/eng.config https://raw.githubusercontent.com/tesseract-ocr/langdata_lstm/main/eng/eng.config
fi

# Download radical-stroke.txt if not present
if [ ! -f /content/drive/MyDrive/langdata_lstm/radical-stroke.txt ]; then
    echo "Downloading radical-stroke.txt..."
    wget -O /content/drive/MyDrive/langdata_lstm/radical-stroke.txt https://raw.githubusercontent.com/tesseract-ocr/langdata_lstm/main/radical-stroke.txt
fi

rm -rf train/*

# Run the training script with your parameters
/content/drive/MyDrive/tesseract/src/training/tesstrain.sh --fonts_dir fonts \
    --fontlist 'TypoUpright BT' \
    --lang eng \
    --linedata_only \
    --langdata_dir /content/drive/MyDrive/langdata_lstm \
    --tessdata_dir /content/drive/MyDrive/tesseract/tessdata \
    --save_box_tiff \
    --maxpages 10 \
    --output_dir train



=== Starting training for language 'eng'
[Tue Mar 4 04:03:43 AM UTC 2025] /usr/bin/text2image --fonts_dir=fonts --ptsize 12 --font=TypoUpright BT --outputbase=/tmp/font_tmp.9bwK862EjO/sample_text.txt --text=/tmp/font_tmp.9bwK862EjO/sample_text.txt --fontconfig_tmpdir=/tmp/font_tmp.9bwK862EjO
Rendered page 0 to file /tmp/font_tmp.9bwK862EjO/sample_text.txt.tif

=== Phase I: Generating training images ===
Rendering using TypoUpright BT
[Tue Mar 4 04:03:44 AM UTC 2025] /usr/bin/text2image --fontconfig_tmpdir=/tmp/font_tmp.9bwK862EjO --fonts_dir=fonts --strip_unrenderable_words --leading=32 --xsize=3600 --char_spacing=0.0 --exposure=0 --outputbase=/tmp/eng-2025-03-04.RuE/eng.TypoUpright_BT.exp0 --max_pages=10 --font=TypoUpright BT --ptsize 12 --text=/content/drive/MyDrive/langdata_lstm/eng/eng.training_text
Stripped 3 unrenderable words
Rendered page 0 to file /tmp/eng-2025-03-04.RuE/eng.TypoUpright_BT.exp0.tif
Stripped 8 unrenderable words
Rendered page 1 to file /tmp/eng-2025-03-04.RuE/

In [22]:
%%shell

rm -rf output/*
OMP_THREAD_LIMIT=16 lstmtraining \
	--continue_from eng.lstm \
	--model_output output/font_name \
	--traineddata /content/drive/MyDrive/tesseract/tessdata/eng.traineddata \
	--train_listfile train/eng.training_files.txt \
	--max_iterations 200

Loaded file eng.lstm, unpacking...
Continuing from eng.lstm
Loaded 506/506 lines (1-506) of document train/eng.TypoUpright_BT.exp0.lstmf
2 Percent improvement time=97, best error was 100 @ 0
At iteration 97/100/100, Mean rms=1.437%, delta=3.007%, char train=8.806%, word train=29.774%, skip ratio=0%,  New best char error = 8.806 Transitioned to stage 1 wrote best model:output/font_name8.806_97.checkpoint wrote checkpoint.

2 Percent improvement time=187, best error was 100 @ 0
At iteration 187/200/200, Mean rms=1.305%, delta=2.502%, char train=7.123%, word train=24.981%, skip ratio=0%,  New best char error = 7.123 wrote best model:output/font_name7.123_187.checkpoint wrote checkpoint.

Finished! Error rate = 7.123


In [23]:
%%shell

lstmtraining --stop_training \
	--continue_from output/font_name_checkpoint \
	--traineddata /content/drive/MyDrive/tesseract/tessdata/eng.traineddata \
	--model_output output/font_name.traineddata

Loaded file output/font_name_checkpoint, unpacking...


In [25]:
from google.colab import files
files.download("output/font_name.traineddata")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# run 
# "C:\Program Files\Tesseract-OCR\tesseract.exe" 31.jpg output.txt -l eng
